#### **Propositional Chunking**

Proposition chunking = splitting text into tiny factual statements instead of random fixed-size blocks.

Previously chunks are formed on the basis of number of characters in chunk and overlap.

This can split a sentence randomly from mid or etc.. (overlap is useful)

But what if we can make each chunk factual and self-contained propositions.

#### **Benefits of Propositional Chunking**

- **Granularity** : By breaking the document into small factual propositions, the system allows for highly specific retrieval, making it easier to extract precise answers from large or complex documents.

- **Quality Assurance** : The use of a quality-checking LLM ensures that the generated propositions meet specific standards, improving the reliability of the retrieved information.

- **Flexibility in Retrieval** : The comparison between proposition-based and larger chunk-based retrieval allows for evaluating the trade-offs between granularity and broader context in search results.

#### **WorkFlow**

- Load document
- Apply Chunking 
- Use propositional Chunking (using LLM)
- Create VectorDB
- Pass the query and retrieved context to LLM
- Evaluate the responses for `propositional chunking` and `Normal Chunking`

In [1]:
## sample content of text 
sample_content = """Paul Graham's essay "Founder Mode," published in September 2024, challenges conventional wisdom about scaling startups, arguing that founders should maintain their unique management style rather than adopting traditional corporate practices as their companies grow.
Conventional Wisdom vs. Founder Mode
The essay argues that the traditional advice given to growing companies—hiring good people and giving them autonomy—often fails when applied to startups.
This approach, suitable for established companies, can be detrimental to startups where the founder's vision and direct involvement are crucial. "Founder Mode" is presented as an emerging paradigm that is not yet fully understood or documented, contrasting with the conventional "manager mode" often advised by business schools and professional managers.
Unique Founder Abilities
Founders possess unique insights and abilities that professional managers do not, primarily because they have a deep understanding of their company's vision and culture.
Graham suggests that founders should leverage these strengths rather than conform to traditional managerial practices. "Founder Mode" is an emerging paradigm that is not yet fully understood or documented, with Graham hoping that over time, it will become as well-understood as the traditional manager mode, allowing founders to maintain their unique approach even as their companies scale.
Challenges of Scaling Startups
As startups grow, there is a common belief that they must transition to a more structured managerial approach. However, many founders have found this transition problematic, as it often leads to a loss of the innovative and agile spirit that drove the startup's initial success.
Brian Chesky, co-founder of Airbnb, shared his experience of being advised to run the company in a traditional managerial style, which led to poor outcomes. He eventually found success by adopting a different approach, influenced by how Steve Jobs managed Apple.
Steve Jobs' Management Style
Steve Jobs' management approach at Apple served as inspiration for Brian Chesky's "Founder Mode" at Airbnb. One notable practice was Jobs' annual retreat for the 100 most important people at Apple, regardless of their position on the organizational chart
. This unconventional method allowed Jobs to maintain a startup-like environment even as Apple grew, fostering innovation and direct communication across hierarchical levels. Such practices emphasize the importance of founders staying deeply involved in their companies' operations, challenging the traditional notion of delegating responsibilities to professional managers as companies scale.
"""

#### LLM used 
ChatOllama (llama3.2)

In [2]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model='llama3.2',
    temperature=0,
    verbose=True
)

llm.invoke("Hey What are you doing right now?")

AIMessage(content='I\'m just a language model, I don\'t have personal experiences or emotions like humans do. However, I am currently:\n\n1. Processing your question and generating a response.\n2. Waiting for any additional input from you to continue our conversation.\n3. Running on computer servers, responding to queries from users like you.\n\nIn other words, I\'m always "on" and ready to help with any questions or topics you\'d like to discuss! How about you? What\'s new with you today?', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-12-04T13:50:17.668565Z', 'done': True, 'done_reason': 'stop', 'total_duration': 27525745708, 'load_duration': 3804161916, 'prompt_eval_count': 33, 'prompt_eval_duration': 13287917291, 'eval_count': 101, 'eval_duration': 7353193249, 'logprobs': None, 'model_name': 'llama3.2', 'model_provider': 'ollama'}, id='lc_run--51091f05-9383-43b5-b951-bd54770d0b7f-0', usage_metadata={'input_tokens': 33, 'output_tokens': 101, 'tota

#### Embedding Model 
HuggingFace - Sentence Transformer

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings 

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

text = "This is a test document."
query_result = embeddings.embed_query(text)

# show only the first 100 characters of the stringified vector
print(f"Dimension of embeddings : {len(query_result)}")
print(str(query_result)[:100] + "...")

Dimension of embeddings : 384
[-0.0383385606110096, 0.1234646886587143, -0.02864295430481434, 0.05365273356437683, 0.0088453618809...


In [4]:
## Lets make our test to be of Document class and add some metadata in it
from langchain_core.documents import Document

doc_list = [Document(page_content=sample_content, metadata={"Title": "Paul Graham's Founder Mode Essay", "Source": "https://www.perplexity.ai/page/paul-graham-s-founder-mode-ess-t9TCyvkqRiyMQJWsHr0fnQ"})]

print(doc_list)

[Document(metadata={'Title': "Paul Graham's Founder Mode Essay", 'Source': 'https://www.perplexity.ai/page/paul-graham-s-founder-mode-ess-t9TCyvkqRiyMQJWsHr0fnQ'}, page_content='Paul Graham\'s essay "Founder Mode," published in September 2024, challenges conventional wisdom about scaling startups, arguing that founders should maintain their unique management style rather than adopting traditional corporate practices as their companies grow.\nConventional Wisdom vs. Founder Mode\nThe essay argues that the traditional advice given to growing companies—hiring good people and giving them autonomy—often fails when applied to startups.\nThis approach, suitable for established companies, can be detrimental to startups where the founder\'s vision and direct involvement are crucial. "Founder Mode" is presented as an emerging paradigm that is not yet fully understood or documented, contrasting with the conventional "manager mode" often advised by business schools and professional managers.\nUniq

In [ ]:
## text splitter from langchain
from langchain_text_splitters import RecursiveCharacterTextSplitter

## 